# gen_P12_classify_regression_data

In [ ]:
from core.tool import IoU
import numpy as np
from numpy.random import randint
import cv2
import os
import time

begin=time.time()

WIDER_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\prepare_data\\WIDER_train\\images"
WIDER_spilt_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\prepare_data\\wider_face_split\\wider_face_train_bbx_gt.txt"
negative_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\12\\negative"
positive_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\12\\positive"
par_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\12\\part"
save_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\12"

if not os.path.exists(positive_dir):
    os.makedirs(positive_dir)
if not os.path.exists(par_dir):
    os.makedirs(par_dir)
if not os.path.exists(negative_dir):
    os.makedirs(negative_dir)

f1 = open(os.path.join(save_dir, 'pos_12.txt'), 'w')
f2 = open(os.path.join(save_dir, 'neg_12.txt'), 'w')
f3 = open(os.path.join(save_dir, 'par_12.txt'), 'w')    

with open(WIDER_spilt_dir) as filenames:
    p=0
    neg_idx=0
    pos_idx=0
    par_idx=0
    for line in filenames.readlines():
        line=line.strip().split(' ')
        if(p==0):
            pic_dir=line[0]
            p=1
            boxes=[]
        elif(p==1):
            k=int(line[0])
            p=2
        elif(p==2):
            b=[]            
            k=k-1
            if(k==0):
                p=0                
            for i in range(4):
                b.append(int(line[i]))
            boxes.append(b)
            # format of boxes is [x,y,w,h]
            if(p==0):
                img=cv2.imread(os.path.join(WIDER_dir,pic_dir).replace('/','\\'))
                h,w,c=img.shape
                
                #save num negative pics whose IoU less than 0.3
                num=50
                while(num):
                    size=randint(12,min(w,h)/2)
                    x=randint(0,w-size)
                    y=randint(0,h-size)
                    if(np.max(IoU(np.array([x,y,x+size,y+size]),np.array(boxes)))<0.3):
                        resized_img = cv2.resize(img[y:y+size,x:x+size,:], (12, 12))
                        cv2.imwrite(os.path.join(negative_dir,'neg_%d.jpg'%(neg_idx)),resized_img)
                        f2.write(os.path.join(negative_dir,'neg_%d.jpg'%(neg_idx)) + ' 0\n')
                        neg_idx=neg_idx+1
                        num=num-1       
#                 num=40
                for box in boxes:
                    if((box[0]<0)|(box[1]<0)|(max(box[2],box[3])<20)|(min(box[2],box[3])<=5)): 
                        continue  
                    x1, y1, w1, h1 = box
                    
                    # crop another 5 images near the bounding box if IoU less than 0.3, save as negative samples
                    for i in range(10):
                        size = randint(12, min(w, h) / 2)
                        delta_x = randint(max(-size, -x1), w1)
                        delta_y = randint(max(-size, -y1), h1)
                        nx1 = int(max(0, x1 + delta_x))
                        ny1 = int(max(0, y1 + delta_y))
                        if((nx1 + size > w1)|(ny1 + size > h1)):
                            continue
                        if(np.max(IoU(np.array([nx1,ny1,nx1+size,ny1+size]),np.array(boxes)))<0.3):
                            resized_img = cv2.resize(img[y:y+size,x:x+size,:], (12, 12))
                            cv2.imwrite(os.path.join(negative_dir,'neg_%d.jpg'%(neg_idx)),resized_img)
                            f2.write(os.path.join(negative_dir,'neg_%d.jpg'%(neg_idx)) + ' 0\n')
                            neg_idx=neg_idx+1
                    #save num positive&part face whose IoU more than 0.65||0.4         
                    box_ = np.array(box).reshape(1, -1)
                    for i in range(10):
                        size=randint(np.floor(0.8*min(w1,h1)),np.ceil(1.25*max(w1,h1))+1)
                    
                        delta_w = randint(-w1 * 0.2, w1 * 0.2 + 1)
                        delta_h = randint(-h1 * 0.2, h1 * 0.2 + 1)
                        # random face box
                        nx1 = int(max(x1 + w1 / 2 + delta_w - size / 2, 0))
                        ny1 = int(max(y1 + h1 / 2 + delta_h - size / 2, 0))
                        nx2 = nx1 + size
                        ny2 = ny1 + size
                        
                        if( nx2 > w | ny2 > h):
                            continue 
                            
                        offset_x1 = (x1 - nx1) / float(size)
                        offset_y1 = (y1 - ny1) / float(size)
                        offset_x2 = (x1+w1 - nx2) / float(size)
                        offset_y2 = (y1+h1 - ny2) / float(size)                                

                        if(IoU(np.array([nx1,ny1,nx2,ny2]),box_)>0.65):                     
                            resized_img = cv2.resize(img[ny1:ny2,nx1:nx2,:], (12, 12))
                            cv2.imwrite(os.path.join(positive_dir,'pos_%d.jpg'%(pos_idx)),resized_img)
                            f1.write(os.path.join(positive_dir,'pos_%d.jpg'%(pos_idx)) + ' 1 %.2f %.2f %.2f %.2f\n'%(offset_x1,offset_y1,offset_x2,offset_y2))
                            pos_idx=pos_idx+1   
                            
                        elif(IoU(np.array([nx1,ny1,nx2,ny2]),box_)>0.4):
                            resized_img = cv2.resize(img[ny1:ny2,nx1:nx2,:], (12, 12))
                            cv2.imwrite(os.path.join(par_dir,'par_%d.jpg'%(par_idx)),resized_img)
                            f3.write(os.path.join(par_dir,'par_%d.jpg'%(par_idx)) + ' -1 %.2f %.2f %.2f %.2f\n'%(offset_x1,offset_y1,offset_x2,offset_y2))                           
                            par_idx=par_idx+1 
    print("pics all done,neg_pics %d in total,pos_pics %d in total,par_pics %d in total"%(neg_idx,pos_idx,par_idx))
    
f1.close()
f2.close()
f3.close()  
print(time.time()-begin)
#6841.530851840973

# Onet
# pics all done,neg_pics 845040 in total,pos_pics 484670 in total,par_pics 708282 in total
# 6949.557825803757

# gen_P12_landmark_data

In [ ]:
from core.tool import IoU,flip
import numpy as np
import random
from numpy.random import randint
import cv2
import os
import time

begin=time.time()

lfw_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\prepare_data"
pic_spilt_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\prepare_data\\trainImageList.txt"
landmark_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\12\\landmark"
save_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\12"

if not os.path.exists(landmark_dir):
    os.makedirs(landmark_dir)
    
f4 = open(os.path.join(save_dir, 'land_12.txt'), 'w')

with open(pic_spilt_dir) as filenames:
    land_idx=0
    for line in filenames:
        img_list=[]
        mark_list=[]
        line=line.strip().split(' ')
        img=cv2.imread(os.path.join(lfw_dir,line[0]))
        box=(line[1],line[2],line[3],line[4])
        box=[int(_) for _ in box]
        #format of box is [x,x+w,y,y+h]
        height,weight,channel=img.shape
        landmark=np.zeros((5,2))
        for i in range(5):
            mark=(float(line[5+2*i]),float(line[5+2*i+1]))
            landmark[i]=mark

        facemark=np.zeros((5,2))
        for i in range(5):
            mark=((landmark[i][0]-box[0])/(box[1]-box[0]),(landmark[i][1]-box[2])/(box[3]-box[2]))
            facemark[i]=mark
        img_list.append(cv2.resize(img[box[2]:box[3],box[0]:box[1]], (12, 12)))  
        mark_list.append(facemark.reshape(10))
        
        box_=[box[0],box[2],box[1],box[3]]
        #format of box is [x,y,x+w,y+h]      
        x1,y1,x2,y2=box_
        w=x2-x1+1
        h=y2-y1+1
        
        if((x1<0)|(y1<0)|(max(w,h)<40)|(min(w,h)<=5)): 
            continue          
        num=25
        while(num):
            
            size=randint(np.floor(0.8*min(w,h)),np.ceil(1.25*max(w,h))+1)
            
            delta_w = randint(-w * 0.2, w * 0.2 + 1)
            delta_h = randint(-h * 0.2, h * 0.2 + 1)
            # random face box
            nx1 = int(max(x1 + w / 2 + delta_w - size / 2, 0))
            ny1 = int(max(y1 + h / 2 + delta_h - size / 2, 0))
            nx2 = nx1 + size
            ny2 = ny1 + size 

            if( nx2 > weight | ny2 > height):
                continue               
            
            _box=[x1,y1,w,h]
            _box=np.array(_box).reshape(1,-1)     
            if(IoU(np.array([nx1,ny1,nx2,ny2]),_box)>0.65): 
                facemark=np.zeros((5,2))
                for i in range(5):
                    mark=((landmark[i][0]-nx1)/size,(landmark[i][1]-ny1)/size)
                    facemark[i]=mark  
                img_list.append(cv2.resize(img[ny1:ny2,nx1:nx2,:], (12, 12)))  
                mark_list.append(facemark.reshape(10))
                              
                #mirro
                mirro_mark=facemark.copy()
                if(random.choice([0,1])):
                    img1,mirro_mark=flip(img[ny1:ny2,nx1:nx2,:],mirro_mark)
                    img_list.append(cv2.resize(img1, (12, 12)))  
                    mark_list.append(mirro_mark.reshape(10))  
                    
                num=num-1
        for i in range(len(img_list)):
            
            if np.sum(np.where(mark_list[i] <= 0, 1, 0)) > 0:
                continue

            if np.sum(np.where(mark_list[i] >= 1, 1, 0)) > 0:
                continue
            
            cv2.imwrite(os.path.join(landmark_dir,'land_%d.jpg'%(land_idx)),img_list[i])
            mark=[str(_)for _ in mark_list[i]]
            f4.write(os.path.join(landmark_dir,'land_%d.jpg'%(land_idx)) +' -2 '+' '.join(mark)+'\n')
            land_idx=land_idx+1

f4.close()    
print(time.time()-begin)
# 1642.9541683197021

# gen_P12_TFRecord

In [ ]:
import tensorflow as tf 
import cv2
import random

terms=['neg_12','pos_12','par_12','land_12']

for term in terms:
    
    with tf.python_io.TFRecordWriter("C:\\Users\\friedhelm\\Desktop\\DATA\\12\\%s_train.tfrecords"%(term)) as writer:
        with open(r'C:\Users\friedhelm\Desktop\DATA\12\%s.txt'%(term)) as readlines:
            readlines=[line.strip().split(' ') for line in readlines]
            random.shuffle(readlines)
            for line in readlines:
                print(line[0])
                img=cv2.imread(line[0].replace('/','\\'))
                img_raw = img.tobytes()
                label=int(line[1])
                roi=[0.0]*4               
                landmark=[0.0]*10
                if(len(line)==6):    
                    roi=[float(_) for _ in line[2:6]]   
                if(len(line)==12):
                    landmark=[float(_) for _ in line[2:12]]                  
                example = tf.train.Example(features=tf.train.Features(feature={
                    'img': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
                    "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
                    "roi": tf.train.Feature(float_list=tf.train.FloatList(value=roi)),
                    "landmark": tf.train.Feature(float_list=tf.train.FloatList(value=landmark)),
                }))
                writer.write(example.SerializeToString())  #序列化为字符串            

# tool

In [ ]:
import tensorflow as tf

def read_single_tfrecord(addr,_batch_size,shape):
    
    filename_queue = tf.train.string_input_producer([addr],shuffle=True)

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue) #返回文件名和文件

    features = tf.parse_single_example(serialized_example,
                                   features={
                                   'img':tf.FixedLenFeature([],tf.string),
                                   'label':tf.FixedLenFeature([],tf.int64),                                   
                                   'roi':tf.FixedLenFeature([4],tf.float32),
                                   'landmark':tf.FixedLenFeature([10],tf.float32),
                                   })
    img=tf.decode_raw(features['img'],tf.uint8)
    label=tf.cast(features['label'],tf.int32)
    roi=tf.cast(features['roi'],tf.float32)
    landmark=tf.cast(features['landmark'],tf.float32)
    img = tf.reshape(img, [shape,shape,3])     
    #     img=img_preprocess(img)
    min_after_dequeue = 10000
    batch_size = _batch_size
    capacity = min_after_dequeue + 10 * batch_size
    image_batch, label_batch, roi_batch, landmark_batch = tf.train.shuffle_batch([img,label,roi,landmark], 
                                                        batch_size=batch_size, 
                                                        capacity=capacity, 
                                                        min_after_dequeue=min_after_dequeue,
                                                        num_threads=7)  
    
    label_batch = tf.reshape(label_batch, [batch_size])
    roi_batch = tf.reshape(roi_batch,[batch_size,4])
    landmark_batch = tf.reshape(landmark_batch,[batch_size,10])
    
    return image_batch, label_batch, roi_batch, landmark_batch


In [ ]:
def read_multi_tfrecords(addr,_batch_size,shape):
    
    pos_dir,part_dir,neg_dir,landmark_dir = addr
    pos_batch_size,part_batch_size,neg_batch_size,landmark_batch_size = _batch_size   
    
    pos_image,pos_label,pos_roi,pos_landmark = read_single_tfrecord(pos_dir, pos_batch_size, shape)
    part_image,part_label,part_roi,part_landmark = read_single_tfrecord(part_dir, part_batch_size, shape)
    neg_image,neg_label,neg_roi,neg_landmark = read_single_tfrecord(neg_dir, neg_batch_size, shape)
    landmark_image,landmark_label,landmark_roi,landmark_landmark = read_single_tfrecord(landmark_dir, landmark_batch_size, shape)

    images = tf.concat([pos_image,part_image,neg_image,landmark_image], 0, name="concat/image")
    labels = tf.concat([pos_label,part_label,neg_label,landmark_label],0,name="concat/label")
    rois = tf.concat([pos_roi,part_roi,neg_roi,landmark_roi],0,name="concat/roi")
    landmarks = tf.concat([pos_landmark,part_landmark,neg_landmark,landmark_landmark],0,name="concat/landmark")
    
    return images,labels,rois,landmarks    

In [ ]:
addr=['C:\\Users\\friedhelm\\Desktop\\DATA\\12\\land_12_train.tfrecords',
      'C:\\Users\\friedhelm\\Desktop\\DATA\\12\\neg_12_train.tfrecords',
      'C:\\Users\\friedhelm\\Desktop\\DATA\\12\\pos_12_train.tfrecords',
      'C:\\Users\\friedhelm\\Desktop\\DATA\\12\\par_12_train.tfrecords']
_batch_size=[1,4,1,1]
shape=12
images,labels,rois,landmarks =read_multi_tfrecords(addr,_batch_size,shape)
i=0
with tf.Session() as sess:
    sess.run((tf.global_variables_initializer(),
              tf.local_variables_initializer()))
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess,coord=coord)    
    while(1):
        i=i+1
        if(i%9==1):
            print(sess.run([labels,rois,landmarks]))

# P12_model

In [ ]:
def prelu(inputs):

    with tf.variable_scope('prelu'):
        alphas = tf.get_variable("alphas", shape=inputs.get_shape()[-1], dtype=tf.float32, initializer=tf.constant_initializer(0.25))
        pos = tf.nn.relu(inputs)
        neg = alphas * (inputs-abs(inputs))*0.5
    
    return pos + neg

def conv2d(_input,name,conv_size,conv_stride,bias_size,pad,activation='prelu'):
    
    regularizer=tf.contrib.layers.l2_regularizer(0.0005)
    with tf.variable_scope(name):
        weight=tf.get_variable('weight',conv_size,initializer=tf.truncated_normal_initializer(stddev=0.1))
        bias=tf.get_variable('bias',bias_size,initializer=tf.constant_initializer(0.0))
        weight_loss=regularizer(weight)
        tf.add_to_collection('weight_loss',weight_loss)
        conv=tf.nn.conv2d(_input,weight,strides=conv_stride,padding=pad)
        he=tf.nn.bias_add(conv,bias)
        relu=tf.cond(tf.equal(activation,'prelu'),lambda:prelu(he),lambda:tf.cond(tf.equal(activation,'softmax'),lambda:tf.nn.softmax(he),lambda:he))     
    
    return relu

def pool(_input,name,kernal_size,kernal_stride,pad):
    
    with tf.variable_scope(name):  
        pool=tf.nn.max_pool(_input,ksize=kernal_size,strides=kernal_stride,padding=pad)  
        
    return pool
    
def Pnet_model(x):
    
    conv_1=conv2d(x,'conv_1',[3,3,3,10],[1,1,1,1],[10],'VALID')
    pool_1=pool(conv_1,'pool_1',[1,2,2,1],[1,2,2,1],'SAME')
    conv_2=conv2d(pool_1,'conv_2',[3,3,10,16],[1,1,1,1],[16],'VALID')
    conv_3=conv2d(conv_2,'conv_3',[3,3,16,32],[1,1,1,1],[32],'VALID')
    
    face_label=conv2d(conv_2,'face_label',[1,1,32,2],[1,1,1,1],[2],'VALID','softmax')
    bounding_box=conv2d(conv_2,'bounding_box',[1,1,32,4],[1,1,1,1],[4],'VALID','None')
    landmark_local=conv2d(conv_2,'landmark_local',[1,1,32,10],[1,1,1,1],[10],'VALID','None')        
    
    return face_label, bounding_box ,landmark_local

In [ ]:
def label_loss(pre_label,act_label):
    
    zeros=tf.zeros_like(label,dtype=tf.int64)
    valid_label=tf.where(tf.less(label,0),zeros,label)
    
    size=tf.size(pre_label)
    pre_label=tf.reshape(pre_label,(1,-1))
    column=tf.range(0,size/2)*2
    column_to_stay=column+valid_label
    pre_label=tf.squeeze(tf.gather(pre_label,column_to_stay))
    loss = -tf.log(pre_label+1e-10)    
    
    ones=tf.ones_like(label,dtype=tf.float32)
    valid_colunm = tf.where(label < zeros,zeros,ones)  
    
    num_column=tf.reduce_sum(valid_colunm)
    num=tf.cast(num_column*0.7,dtype=tf.int64)
    loss=loss*valid_colunm
    loss,_=tf.nn.top_k(loss,num)
    
    return loss
    
def box_los(label,pre_box,act_box) :    
    
    zeros=tf.zeros_like(label,dtype=tf.float32)
    ones=tf.ones_like(label,dtype=tf.float32)    
    valid_label=tf.where(tf.equal(abs(label),1),ones,zeros)
    loss=tf.reduce_sum(tf.square(act_box-pre_box),axis=1)
    loss=loss*valid_label
    return tf.reduce_mean(loss)  
    
def landmark_los(label,landmark,_landmark):    
    
    zeros=tf.zeros_like(label,dtype=tf.float32)
    ones = tf.ones_like(label,dtype=tf.float32)
    valid_label=tf.where(tf.equal(label,-2),ones,zeros)
    loss=tf.reduce_sum(tf.square(_landmark-landmark),axis=1)
    loss=loss*valid_label
    return tf.reduce_mean(loss)     
    
def cal_accuracy(cls_prob,label):

    pred = tf.argmax(cls_prob,axis=1)
    label_int = tf.cast(label,tf.int64)
    cond = tf.where(tf.greater_equal(label_int,0))
    picked = tf.squeeze(cond)
    label_picked = tf.gather(label_int,picked)
    pred_picked = tf.gather(pred,picked)
    accuracy_op = tf.reduce_mean(tf.cast(tf.equal(label_picked,pred_picked),tf.float32))
    
    return accuracy_op

# train_P12model

In [ ]:
import tensorflow as tf
import numpy as np
import time
import cv2
from core.tool import read_multi_tfrecords
from core.MTCNN_model import Pnet_model



def image_color_distort(inputs):
    inputs = tf.image.random_contrast(inputs, lower=0.5, upper=1.5)
    inputs = tf.image.random_brightness(inputs, max_delta=0.2)
    inputs = tf.image.random_hue(inputs,max_delta= 0.2)
    inputs = tf.image.random_saturation(inputs,lower = 0.5, upper= 1.5)

    return inputs

def label_los(pre_label,act_label):
    
    ratio=tf.constant(0.7)
    zeros=tf.zeros_like(act_label,dtype=tf.int32)
    valid_label=tf.where(tf.less(act_label,0),zeros,act_label)

    column_num=tf.shape(pre_label,out_type=tf.int32)[0]
    pre_label=tf.squeeze(tf.reshape(pre_label,(1,-1)))
    column=tf.range(0,column_num)*2 
    column_to_stay=column+valid_label

    pre_label=tf.squeeze(tf.gather(pre_label,column_to_stay))
    loss = -tf.log(pre_label+1e-10)      
    ones=tf.ones_like(act_label,dtype=tf.float32)
    zero=tf.zeros_like(act_label,dtype=tf.float32)
    valid_colunm = tf.where(act_label < zeros,zero,ones)  
    
    num_column=tf.reduce_sum(valid_colunm)
    num=tf.cast(num_column*ratio,dtype=tf.int32)
    loss=tf.multiply(loss,valid_colunm,'label_los')
    loss,_=tf.nn.top_k(loss,num)
    
    return tf.reduce_mean(loss)
    
def roi_los(label,pre_box,act_box) :    
    
    zeros=tf.zeros_like(label,dtype=tf.float32)
    ones=tf.ones_like(label,dtype=tf.float32)    
    valid_label=tf.where(tf.equal(abs(label),1),ones,zeros)
    loss=tf.reduce_sum(tf.square(act_box-pre_box),axis=1)
    loss=tf.multiply(loss,valid_label,'roi_los')
    return tf.reduce_mean(loss) 
    
def landmark_los(label,pre_landmark,act_landmark):    
    
    zeros=tf.zeros_like(label,dtype=tf.float32)
    ones = tf.ones_like(label,dtype=tf.float32)
    valid_label=tf.where(tf.equal(label,-2),ones,zeros)
    loss=tf.reduce_sum(tf.square(act_landmark-pre_landmark),axis=1)
    loss=tf.multiply(loss,valid_label,'landmark_los')
    return tf.reduce_mean(loss)     
    
def cal_accuracy(cls_prob,label):
       
    pred = tf.argmax(cls_prob,axis=1)
    label_int = tf.cast(label,tf.int64)
    cond = tf.where(tf.greater_equal(label_int,0))
    picked = tf.squeeze(cond)
    label_picked = tf.gather(label_int,picked)
    pred_picked = tf.gather(pred,picked)
    accuracy_op = tf.reduce_mean(tf.cast(tf.equal(label_picked,pred_picked),tf.float32))
    
    return accuracy_op

def train(image,label,roi,landmark,model,model_name):
    
    _label, _roi ,_landmark=model(image)
    
    with tf.name_scope('output'):
        _label=tf.squeeze(_label,name='label')
        _roi=tf.squeeze(_roi,name='roi')
        _landmark=tf.squeeze(_landmark,name='landmark')
        
    _label_los=label_los(_label,label)
    _box_los=roi_los(label,_roi,roi)    
    _landmark_los=landmark_los(label,_landmark,landmark)
    
    function_loss=_label_los+_box_los*0.5+_landmark_los*0.5

    tf.add_to_collection("loss", function_loss)
    loss_all=tf.get_collection('loss')
    
    with tf.name_scope('loss'):
        loss=tf.reduce_sum(loss_all)
        tf.summary.scalar('loss',loss) 
        
    opt=tf.train.AdamOptimizer(0.001).minimize(loss)
    
    with tf.name_scope('accuracy'):
        train_accuracy=cal_accuracy(_label,label)
        tf.summary.scalar('accuracy',train_accuracy) 

    saver=tf.train.Saver()
    merged=tf.summary.merge_all() 
    
    images,labels,rois,landmarks=read_multi_tfrecords(addr,batch_size,img_size)   
    images=image_color_distort(images)    
    with tf.Session() as sess:
        sess.run((tf.global_variables_initializer(),
                  tf.local_variables_initializer()))
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess,coord=coord)
        image_batch,label_batch,roi_batch,landmark_batch=sess.run([images,labels,rois,landmarks])
    
        writer_train=tf.summary.FileWriter('C:\\Users\\312\\Desktop\\',sess.graph)
        
        for i in range(1,100001):
            
            image_batch,label_batch,roi_batch,landmark_batch=sess.run([images,labels,rois,landmarks])

            sess.run(opt,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch})
            if(i%100==0):
                summary=sess.run(merged,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch})
                writer_train.add_summary(summary,i) 
            if(i%1000==0):
                print('次数',i)    
                print('train_accuracy',sess.run(train_accuracy,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))
                print('loss',sess.run(loss,{image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))               
                print('time',time.time()-begin)
                if(i%10000==0):
                    saver.save(sess,"E:\\friedhelm\\object\\face_detection_MTCNN\\model\\%s.ckpt"%(model_name),global_step=i)
    writer_train.close()
    
def main(model,model_name):
    
    with tf.name_scope('input'):
        image=tf.placeholder(tf.float32,name='image')
        label=tf.placeholder(tf.int32,name='label')
        roi=tf.placeholder(tf.float32,name='roi')
        landmark = tf.placeholder(tf.float32,name='landmark')  

    train(image,label,roi,landmark,model,model_name)

if __name__=='__main__':
    
    img_size=12
    batch=448
    batch_size=[192,64,64,128]
    addr=["E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\neg_%d_train.tfrecords"%(img_size,img_size),
          "E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\pos_%d_train.tfrecords"%(img_size,img_size),
          "E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\par_%d_train.tfrecords"%(img_size,img_size),
          "E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\land_%d_train.tfrecords"%(img_size,img_size)]  
        
    model=Pnet_model
    model_name="Pnet_model"    
    
    begin=time.time()
    
    main(model,model_name)
# tensorboard --logdir=C:\\Users\\312\\Desktop\\

# gen_R24_classify_regression_data

In [ ]:
#159424张人脸
import tensorflow as tf 
import numpy as np
import cv2
import time
from core.tool import IoU,NMS,featuremap
import os

begin=time.time()
img_size=24
map_shape=12
stride=2
factor=0.79
threshold=0.8

graph_path='E:\\friedhelm\\object\\face_detection_MTCNN\\\model\\Pnet_model.ckpt-60000.meta'
model_path='E:\\friedhelm\\object\\face_detection_MTCNN\\\model\\Pnet_model.ckpt-60000'
saver=tf.train.import_meta_graph(graph_path)

WIDER_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\prepare_data\\WIDER_train\\images"
WIDER_spilt_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\prepare_data\\wider_face_split\\wider_face_train_bbx_gt.txt"
negative_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\negative"%(img_size)
positive_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\positive"%(img_size)
par_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\part"%(img_size)
save_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d"%(img_size)

if not os.path.exists(positive_dir):
    os.makedirs(positive_dir)
if not os.path.exists(par_dir):
    os.makedirs(par_dir)
if not os.path.exists(negative_dir):
    os.makedirs(negative_dir)
    
f1 = open(os.path.join(save_dir, 'pos_%d.txt'%(img_size)), 'w')
f2 = open(os.path.join(save_dir, 'neg_%d.txt'%(img_size)), 'w')
f3 = open(os.path.join(save_dir, 'par_%d.txt'%(img_size)), 'w')   

with tf.Session() as sess: 
    
    saver.restore(sess,model_path)
    graph = tf.get_default_graph()

    with open(WIDER_spilt_dir) as filenames:
        p=0
        idx=0
        neg_idx=0
        pos_idx=0
        par_idx=0
        for line in filenames.readlines():
            line=line.strip().split(' ')
            if(p==0):
                pic_dir=line[0]
                p=1
                boxes=[]
            elif(p==1):
                k=int(line[0])
                p=2
            elif(p==2):
                b=[]            
                k=k-1
                if(k==0):
                    p=0                
                for i in range(4):
                    b.append(int(line[i]))
                boxes.append(b)
                # format of boxes is [x,y,w,h]
                if(p==0):
                    img=cv2.imread(os.path.join(WIDER_dir,pic_dir).replace('/','\\'))
                    h,w,c=img.shape
                    print(pic_dir)
                    if(min(h,w)<20):
                        continue
                        
                    scales=[]
                    total_box=[]
#                     bounding_boxes=[]
#                     NMS_box=[]
                    small=min(img.shape[0:2])
                    pro=12/20
        
                    while small>=20:
                        scales.append(pro)
                        pro*=factor
                        small*=factor  

                    for scale in scales:

                        scale_img=cv2.resize(img,((int(img.shape[1]*scale)),(int(img.shape[0]*scale))))
                        bounding_boxes=featuremap(sess,graph,scale_img,scale,map_shape,stride,threshold)

                        if(bounding_boxes):
                            for box in bounding_boxes:
                                total_box.append(box)
                                                  
                    NMS_box=NMS(total_box)
                    neg_num=0
                    for box_ in NMS_box:
                        
                        box=box_.copy()                        
                        if((box[0]<0)|(box[1]<0)|(box[2]>w)|(box[3]>h)|(box[2]-box[0]<=20)|(box[3]-box[1]<=20)): 
                            continue  
                        # format of total_box: [x1,y1,x2,y2,score,offset_x1,offset_y1,offset_x2,offset_y2,10*landmark]  
                        
                        t_box=[0]*4
                        t_w=box[2]-box[0]+1
                        t_h=box[3]-box[1]+1
                        
                        t_box[0]=box[5]*t_w+box[0]
                        t_box[1]=box[6]*t_h+box[1]                     
                        t_box[2]=box[7]*t_w+box[2]    
                        t_box[3]=box[8]*t_h+box[3]                        
                        # 计算真实人脸框
                        
                        if((t_box[0]<0)|(t_box[1]<0)|(t_box[2]>img.shape[1])|(t_box[3]>img.shape[0])|(t_box[2]-t_box[0]<=20)|(t_box[3]-t_box[1]<=20)): 
                            continue 
                            
                        ti_box=t_box.copy()
                        ti_box=[int(_) for _ in ti_box]
                        
                        Iou=IoU(np.array(t_box),np.array(boxes))
                        
                        if((np.max(Iou)<0.3)&(neg_num<60)):
                            resized_img = cv2.resize(img[ti_box[1]:ti_box[3],ti_box[0]:ti_box[2],:], (img_size,img_size))
                            cv2.imwrite(os.path.join(negative_dir,'neg_%d.jpg'%(neg_idx)),resized_img)
                            f2.write(os.path.join(negative_dir,'neg_%d.jpg'%(neg_idx)) + ' 0\n')
                            neg_idx=neg_idx+1
                            neg_num=neg_num+1
                        
                        else:
                            x1,y1,w1,h1=boxes[np.argmax(Iou)]
                            
                            offset_x1 = (x1 - t_box[0]) / float(t_box[2]-t_box[0]+1)
                            offset_y1 = (y1 - t_box[1]) / float(t_box[3]-t_box[1]+1)
                            offset_x2 = (x1+w1 - t_box[2]) / float(t_box[2]-t_box[0]+1)
                            offset_y2 = (y1+h1 - t_box[3]) / float(t_box[3]-t_box[1]+1)                         
                            
                            if(np.max(Iou)>0.65):                    
                                resized_img = cv2.resize(img[ti_box[1]:ti_box[3],ti_box[0]:ti_box[2],:], (img_size, img_size))
                                cv2.imwrite(os.path.join(positive_dir,'pos_%d.jpg'%(pos_idx)),resized_img)
                                f1.write(os.path.join(positive_dir,'pos_%d.jpg'%(pos_idx)) + ' 1 %.2f %.2f %.2f %.2f\n'%(offset_x1,offset_y1,offset_x2,offset_y2))
                                pos_idx=pos_idx+1   

                            elif(np.max(Iou)>0.4):
                                resized_img = cv2.resize(img[ti_box[1]:ti_box[3],ti_box[0]:ti_box[2],:], (img_size, img_size))
                                cv2.imwrite(os.path.join(par_dir,'par_%d.jpg'%(par_idx)),resized_img)
                                f3.write(os.path.join(par_dir,'par_%d.jpg'%(par_idx)) + ' -1 %.2f %.2f %.2f %.2f\n'%(offset_x1,offset_y1,offset_x2,offset_y2))                           
                                par_idx=par_idx+1 
                    idx+=1
                    if(idx%100==0):
                        print('idx: ',idx," ;neg_idx: ",neg_idx," ;pos_idx: ",pos_idx," ;par_idx: ",par_idx)
                        print(time.time()-begin)
    print("pics all done,neg_pics %d in total,pos_pics %d in total,par_pics %d in total"%(neg_idx,pos_idx,par_idx))
    
f1.close()
f2.close()
f3.close()  
print(time.time()-begin)

#pics all done,neg_pics 758503 in total,pos_pics 285017 in total,par_pics 572771 in total
# 17590.795156002045

# gen_R24_landmark_data

In [ ]:
from core.tool import IoU,flip
import numpy as np
import random
from numpy.random import randint
import cv2
import os
import time

def main():
    
    f4 = open(os.path.join(save_dir, 'land_%d.txt'%(img_size)), 'w')

    with open(pic_spilt_dir) as filenames:
        land_idx=0
        for line in filenames:
            img_list=[]
            mark_list=[]
            line=line.strip().split(' ')
            img=cv2.imread(os.path.join(lfw_dir,line[0]))
            box=(line[1],line[2],line[3],line[4])
            box=[int(_) for _ in box]
            #format of box is [x,x+w,y,y+h]
            height,weight,channel=img.shape
            landmark=np.zeros((5,2))
            for i in range(5):
                mark=(float(line[5+2*i]),float(line[5+2*i+1]))
                landmark[i]=mark

            facemark=np.zeros((5,2))
            for i in range(5):
                mark=((landmark[i][0]-box[0])/(box[1]-box[0]),(landmark[i][1]-box[2])/(box[3]-box[2]))
                facemark[i]=mark
            img_list.append(cv2.resize(img[box[2]:box[3],box[0]:box[1]], (img_size, img_size)))  
            mark_list.append(facemark.reshape(10))

            box_=[box[0],box[2],box[1],box[3]]
            #format of box is [x,y,x+w,y+h]      
            x1,y1,x2,y2=box_
            w=x2-x1+1
            h=y2-y1+1

            if((x1<0)|(y1<0)|(max(w,h)<40)|(min(w,h)<=5)): 
                continue          
            num=40
            while(num):

                size=randint(np.floor(0.8*min(w,h)),np.ceil(1.25*max(w,h))+1)

                delta_w = randint(-w * 0.2, w * 0.2 + 1)
                delta_h = randint(-h * 0.2, h * 0.2 + 1)
                # random face box
                nx1 = int(max(x1 + w / 2 + delta_w - size / 2, 0))
                ny1 = int(max(y1 + h / 2 + delta_h - size / 2, 0))
                nx2 = nx1 + size
                ny2 = ny1 + size 

                if( nx2 > weight | ny2 > height):
                    continue               

                _box=[x1,y1,w,h]
                _box=np.array(_box).reshape(1,-1)     
                if(IoU(np.array([nx1,ny1,nx2,ny2]),_box)>0.65): 
                    facemark=np.zeros((5,2))
                    for i in range(5):
                        mark=((landmark[i][0]-nx1)/size,(landmark[i][1]-ny1)/size)
                        facemark[i]=mark  
                    img_list.append(cv2.resize(img[ny1:ny2,nx1:nx2,:], (img_size, img_size)))  
                    mark_list.append(facemark.reshape(10))

                    #mirro
                    mirro_mark=facemark.copy()
                    if(random.choice([0,1])):
                        img1,mirro_mark=flip(img[ny1:ny2,nx1:nx2,:],mirro_mark)
                        img_list.append(cv2.resize(img1, (img_size, img_size)))  
                        mark_list.append(mirro_mark.reshape(10))  

                    num=num-1
            for i in range(len(img_list)):

                if np.sum(np.where(mark_list[i] <= 0, 1, 0)) > 0:
                    continue

                if np.sum(np.where(mark_list[i] >= 1, 1, 0)) > 0:
                    continue

                cv2.imwrite(os.path.join(landmark_dir,'land_%d.jpg'%(land_idx)),img_list[i])
                mark=[str(_)for _ in mark_list[i]]
                f4.write(os.path.join(landmark_dir,'land_%d.jpg'%(land_idx)) +' -2 '+' '.join(mark)+'\n')
                land_idx=land_idx+1

    f4.close()    

if __name__=="__main__":

    img_size=24
    
    #change img_size to P R O net
    begin=time.time()

    lfw_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\prepare_data"
    pic_spilt_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\prepare_data\\trainImageList.txt"
    landmark_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\landmark"%(img_size)
    save_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d"%(img_size)

    if not os.path.exists(landmark_dir):
        os.makedirs(landmark_dir)    

    main()
    
    print(time.time()-begin)
    
#R_net 2320.5425374507904
#O_net 3131.904032945633

# gen_R24_TFRecord

In [ ]:
import tensorflow as tf 
import cv2
import random
import time

def main():
    
    for index,term in enumerate(terms):
        num=0
        print("%s start"%(term))
        with tf.python_io.TFRecordWriter("E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\%s_train.tfrecords"%(img_size,term)) as writer:
            with open(r'E:\friedhelm\object\face_detection_MTCNN\DATA\%d\%s.txt'%(img_size,term)) as readlines:
                readlines=[line.strip().split(' ') for line in readlines]
                random.shuffle(readlines)
                for i,line in enumerate(readlines):
                    if(i%50000==0):
                        print(i)
                    img=cv2.imread(line[0].replace('/','\\'))
                    img_raw = img.tobytes()
                    label=int(line[1])
                    roi=[0.0]*4               
                    landmark=[0.0]*10
                    if(len(line)==6):    
                        roi=[float(_) for _ in line[2:6]]   
                    if(len(line)==12):
                        landmark=[float(_) for _ in line[2:12]]                  
                    example = tf.train.Example(features=tf.train.Features(feature={
                        'img': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
                        "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
                        "roi": tf.train.Feature(float_list=tf.train.FloatList(value=roi)),
                        "landmark": tf.train.Feature(float_list=tf.train.FloatList(value=landmark)),
                    }))
                    writer.write(example.SerializeToString())  #序列化为字符串  
                    num+=1
                    if(num==base*scale[index]):
                        print("%s finish"%(term))
                        break

if __name__=="__main__":
    
    img_size=24
    #change img_size to P R O net
    terms=['neg_%d'%(img_size),'pos_%d'%(img_size),'par_%d'%(img_size),'land_%d'%(img_size)]
    scale=[3,1,1,2]
    base=200000

    begin=time.time()

    main()
    
    print(time.time()-begin)

#Pnet train_data               
#neg 645017
#par 507206
#pos 285560 
#land 584332
    
#Rnet train_data neg_pics 758503 in total,pos_pics 285017 in total,par_pics 572771 in total land 

#29951.208035707474

# train_R24model

In [ ]:
import tensorflow as tf
import time
from core.tool import read_multi_tfrecords
from MTCNN_model import Pnet_model,Rnet_model,Onet_model
import os

def image_color_distort(inputs):
    inputs = tf.image.random_contrast(inputs, lower=0.5, upper=1.5)
    inputs = tf.image.random_brightness(inputs, max_delta=0.2)
    inputs = tf.image.random_hue(inputs,max_delta= 0.2)
    inputs = tf.image.random_saturation(inputs,lower = 0.5, upper= 1.5)

    return inputs

def label_los(pre_label,act_label):
    
    ratio=tf.constant(0.7)
    zeros=tf.zeros_like(act_label,dtype=tf.int32)
    valid_label=tf.where(tf.less(act_label,0),zeros,act_label)

    column_num=tf.shape(pre_label,out_type=tf.int32)[0]
    pre_label=tf.squeeze(tf.reshape(pre_label,(1,-1)))
    column=tf.range(0,column_num)*2 
    column_to_stay=column+valid_label

    pre_label=tf.squeeze(tf.gather(pre_label,column_to_stay))
    loss = -tf.log(pre_label+1e-10)      
    ones=tf.ones_like(act_label,dtype=tf.float32)
    zero=tf.zeros_like(act_label,dtype=tf.float32)
    valid_colunm = tf.where(act_label < zeros,zero,ones)  
    
    num_column=tf.reduce_sum(valid_colunm)
    num=tf.cast(num_column*ratio,dtype=tf.int32)
    loss=tf.multiply(loss,valid_colunm,'label_los')
    loss,_=tf.nn.top_k(loss,num)
    
    return tf.reduce_mean(loss)
    
def roi_los(label,pre_box,act_box) :    
    
    zeros=tf.zeros_like(label,dtype=tf.float32)
    ones=tf.ones_like(label,dtype=tf.float32)    
    valid_label=tf.where(tf.equal(abs(label),1),ones,zeros)
    loss=tf.reduce_sum(tf.square(act_box-pre_box),axis=1)
    loss=tf.multiply(loss,valid_label,'roi_los')
    return tf.reduce_mean(loss) 
    
def landmark_los(label,pre_landmark,act_landmark):    
    
    zeros=tf.zeros_like(label,dtype=tf.float32)
    ones = tf.ones_like(label,dtype=tf.float32)
    valid_label=tf.where(tf.equal(label,-2),ones,zeros)
    loss=tf.reduce_sum(tf.square(act_landmark-pre_landmark),axis=1)
    loss=tf.multiply(loss,valid_label,'landmark_los')
    return tf.reduce_mean(loss)     
    
def cal_accuracy(cls_prob,label):
       
    pred = tf.argmax(cls_prob,axis=1)
    label_int = tf.cast(label,tf.int64)
    cond = tf.where(tf.greater_equal(label_int,0))
    picked = tf.squeeze(cond)
    label_picked = tf.gather(label_int,picked)
    pred_picked = tf.gather(pred,picked)
    accuracy_op = tf.reduce_mean(tf.cast(tf.equal(label_picked,pred_picked),tf.float32))
    
    return accuracy_op

def train(image,label,roi,landmark,model,model_name):
    
    _label, _roi ,_landmark=model(image)
    
    with tf.name_scope('output'):
        _label=tf.squeeze(_label,name='label')
        _roi=tf.squeeze(_roi,name='roi')
        _landmark=tf.squeeze(_landmark,name='landmark')
        
    _label_los=label_los(_label,label)
    _box_los=roi_los(label,_roi,roi)    
    _landmark_los=landmark_los(label,_landmark,landmark)
    
    function_loss=_label_los*ratio[0]+_box_los*ratio[1]+_landmark_los*ratio[2]

    tf.add_to_collection("loss", function_loss)
    loss_all=tf.get_collection('loss')
    
    with tf.name_scope('loss'):
        loss=tf.reduce_sum(loss_all)
        tf.summary.scalar('loss',loss) 
        
    opt=tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
    with tf.name_scope('accuracy'):
        train_accuracy=cal_accuracy(_label,label)
        tf.summary.scalar('accuracy',train_accuracy) 

    saver=tf.train.Saver()
    merged=tf.summary.merge_all() 
    
    images,labels,rois,landmarks=read_multi_tfrecords(addr,batch_size,img_size)   
    images=image_color_distort(images)
    
    with tf.Session() as sess:
        sess.run((tf.global_variables_initializer(),
                  tf.local_variables_initializer()))
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess,coord=coord)
        image_batch,label_batch,roi_batch,landmark_batch=sess.run([images,labels,rois,landmarks])
        
        writer_train=tf.summary.FileWriter('C:\\Users\\312\\Desktop\\',sess.graph)
        try:
            
            for i in range(1,train_step):
                
                image_batch,label_batch,roi_batch,landmark_batch=sess.run([images,labels,rois,landmarks])
                
                sess.run(opt,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch})
                if(i%100==0):
                    summary=sess.run(merged,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch})
                    writer_train.add_summary(summary,i) 
                if(i%1000==0):
                    print('次数',i)    
                    print('train_accuracy',sess.run(train_accuracy,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))
                    print('loss',sess.run(loss,{image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))               
                    print('time',time.time()-begin)
                    if(i%10000==0):
                        saver.save(sess,"E:\\friedhelm\\object\\face_detection_MTCNN\\%s\\%s_%d.ckpt"%(model_name,model_name,i))
        except  tf.errors.OutOfRangeError:
            print("finished")
        finally:
            coord.request_stop()
            writer_train.close()
        coord.join(threads)
    
def main(model):
    
    with tf.name_scope('input'):
        image=tf.placeholder(tf.float32,name='image')
        label=tf.placeholder(tf.int32,name='label')
        roi=tf.placeholder(tf.float32,name='roi')
        landmark = tf.placeholder(tf.float32,name='landmark')  

    train(image,label,roi,landmark,model,model_name)

if __name__=='__main__':
    
    img_size=24
    batch=448
    batch_size=[192,64,64,128]
    addr=["E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\neg_%d_train.tfrecords"%(img_size),
          "E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\pos_%d_train.tfrecords"%(img_size),
          "E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\par_%d_train.tfrecords"%(img_size),
          "E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\land_%d_train.tfrecords"%(img_size)]  

    model=Rnet_model
    model_name="Rnet_model"    
    train_step=100001
    learning_rate=0.001
    
    save_model_path="E:\\friedhelm\\object\\face_detection_MTCNN\\%s"%(model_name)
    
    if not os.path.exists(save_model_path):
        os.makedirs(save_model_path) 
        
    if(model_name=="Onet_model"):
        ratio=[1,0.5,1]
    else:
        ratio=[1,0.5,0.5]
    

    begin=time.time()        
    main(model)
# tensorboard --logdir=C:\\Users\\312\\Desktop\\

# gen_O48_classify_regression_data

In [ ]:
from core.tool import IoU
import numpy as np
from numpy.random import randint
import cv2
import os
import time

def main():

    f1 = open(os.path.join(save_dir, 'pos_%d.txt'%(img_size)), 'w')
    f2 = open(os.path.join(save_dir, 'neg_%d.txt'%(img_size)), 'w')
    f3 = open(os.path.join(save_dir, 'par_%d.txt'%(img_size)), 'w')    
    
    with open(WIDER_spilt_dir) as filenames:
        p=0
        neg_idx=0
        pos_idx=0
        par_idx=0
        for line in filenames.readlines():
            line=line.strip().split(' ')
            if(p==0):
                pic_dir=line[0]
                p=1
                boxes=[]
            elif(p==1):
                k=int(line[0])
                p=2
            elif(p==2):
                b=[]            
                k=k-1
                if(k==0):
                    p=0                
                for i in range(4):
                    b.append(int(line[i]))
                boxes.append(b)
                # format of boxes is [x,y,w,h]
                if(p==0):
                    img=cv2.imread(os.path.join(WIDER_dir,pic_dir).replace('/','\\'))
                    h,w,c=img.shape
                    
                    #save num negative pics whose IoU less than 0.3
                    num=50
                    while(num):
                        size=randint(12,min(w,h)/2)
                        x=randint(0,w-size)
                        y=randint(0,h-size)
                        if(np.max(IoU(np.array([x,y,x+size,y+size]),np.array(boxes)))<0.3):
                            resized_img = cv2.resize(img[y:y+size,x:x+size,:], (img_size, img_size))
                            cv2.imwrite(os.path.join(negative_dir,'neg_%d.jpg'%(neg_idx)),resized_img)
                            f2.write(os.path.join(negative_dir,'neg_%d.jpg'%(neg_idx)) + ' 0\n')
                            neg_idx=neg_idx+1
                            num=num-1       

                    for box in boxes:
                        if((box[0]<0)|(box[1]<0)|(max(box[2],box[3])<20)|(min(box[2],box[3])<=5)): 
                            continue  
                        x1, y1, w1, h1 = box
                        
                        # crop images near the bounding box if IoU less than 0.3, save as negative samples
                        for i in range(10):
                            size = randint(12, min(w, h) / 2)
                            delta_x = randint(max(-size, -x1), w1)
                            delta_y = randint(max(-size, -y1), h1)
                            nx1 = int(max(0, x1 + delta_x))
                            ny1 = int(max(0, y1 + delta_y))
                            if((nx1 + size > w1)|(ny1 + size > h1)):
                                continue
                            if(np.max(IoU(np.array([nx1,ny1,nx1+size,ny1+size]),np.array(boxes)))<0.3):
                                resized_img = cv2.resize(img[y:y+size,x:x+size,:], (img_size, img_size))
                                cv2.imwrite(os.path.join(negative_dir,'neg_%d.jpg'%(neg_idx)),resized_img)
                                f2.write(os.path.join(negative_dir,'neg_%d.jpg'%(neg_idx)) + ' 0\n')
                                neg_idx=neg_idx+1
                                
                        #save num positive&part face whose IoU more than 0.65|0.4         
                        box_ = np.array(box).reshape(1, -1)
                        for i in range(10):
                            size=randint(np.floor(0.8*min(w1,h1)),np.ceil(1.25*max(w1,h1))+1)
                        
                            delta_w = randint(-w1 * 0.2, w1 * 0.2 + 1)
                            delta_h = randint(-h1 * 0.2, h1 * 0.2 + 1)
                            # random face box
                            nx1 = int(max(x1 + w1 / 2 + delta_w - size / 2, 0))
                            ny1 = int(max(y1 + h1 / 2 + delta_h - size / 2, 0))
                            nx2 = nx1 + size
                            ny2 = ny1 + size
                            
                            if( nx2 > w | ny2 > h):
                                continue 
                                
                            offset_x1 = (x1 - nx1) / float(size)
                            offset_y1 = (y1 - ny1) / float(size)
                            offset_x2 = (x1+w1 - nx2) / float(size)
                            offset_y2 = (y1+h1 - ny2) / float(size)                                
    
                            if(IoU(np.array([nx1,ny1,nx2,ny2]),box_)>0.65):                     
                                resized_img = cv2.resize(img[ny1:ny2,nx1:nx2,:], (img_size, img_size))
                                cv2.imwrite(os.path.join(positive_dir,'pos_%d.jpg'%(pos_idx)),resized_img)
                                f1.write(os.path.join(positive_dir,'pos_%d.jpg'%(pos_idx)) + ' 1 %.2f %.2f %.2f %.2f\n'%(offset_x1,offset_y1,offset_x2,offset_y2))
                                pos_idx=pos_idx+1   
                                
                            elif(IoU(np.array([nx1,ny1,nx2,ny2]),box_)>0.4):
                                resized_img = cv2.resize(img[ny1:ny2,nx1:nx2,:], (img_size, img_size))
                                cv2.imwrite(os.path.join(par_dir,'par_%d.jpg'%(par_idx)),resized_img)
                                f3.write(os.path.join(par_dir,'par_%d.jpg'%(par_idx)) + ' -1 %.2f %.2f %.2f %.2f\n'%(offset_x1,offset_y1,offset_x2,offset_y2))                           
                                par_idx=par_idx+1 
        print("pics all done,neg_pics %d in total,pos_pics %d in total,par_pics %d in total"%(neg_idx,pos_idx,par_idx))
        
    f1.close()
    f2.close()
    f3.close()  


if __name__=="__main__":
    
    img_size=48
    
    WIDER_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\prepare_data\\WIDER_train\\images"
    WIDER_spilt_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\prepare_data\\wider_face_split\\wider_face_train_bbx_gt.txt"
    negative_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\negative"%(img_size)
    positive_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\positive"%(img_size)
    par_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\part"%(img_size)
    save_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d"%(img_size)
    
    if not os.path.exists(positive_dir):
        os.makedirs(positive_dir)
    if not os.path.exists(par_dir):
        os.makedirs(par_dir)
    if not os.path.exists(negative_dir):
        os.makedirs(negative_dir) 
        
    begin=time.time()
    
    main()

    print(time.time()-begin)
#6841.530851840973

# gen_O48_landmark_data

In [ ]:
from core.tool import IoU,flip
import numpy as np
import random
from numpy.random import randint
import cv2
import os
import time

def main():
    
    f4 = open(os.path.join(save_dir, 'land_%d.txt'%(img_size)), 'w')

    with open(pic_spilt_dir) as filenames:
        land_idx=0
        for line in filenames:
            img_list=[]
            mark_list=[]
            line=line.strip().split(' ')
            img=cv2.imread(os.path.join(lfw_dir,line[0]))
            box=(line[1],line[2],line[3],line[4])
            box=[int(_) for _ in box]
            #format of box is [x,x+w,y,y+h]
            height,weight,channel=img.shape
            landmark=np.zeros((5,2))
            for i in range(5):
                mark=(float(line[5+2*i]),float(line[5+2*i+1]))
                landmark[i]=mark

            facemark=np.zeros((5,2))
            for i in range(5):
                mark=((landmark[i][0]-box[0])/(box[1]-box[0]),(landmark[i][1]-box[2])/(box[3]-box[2]))
                facemark[i]=mark
            img_list.append(cv2.resize(img[box[2]:box[3],box[0]:box[1]], (img_size, img_size)))  
            mark_list.append(facemark.reshape(10))

            box_=[box[0],box[2],box[1],box[3]]
            #format of box is [x,y,x+w,y+h]      
            x1,y1,x2,y2=box_
            w=x2-x1+1
            h=y2-y1+1

            if((x1<0)|(y1<0)|(max(w,h)<40)|(min(w,h)<=5)): 
                continue          
            num=40
            while(num):

                size=randint(np.floor(0.8*min(w,h)),np.ceil(1.25*max(w,h))+1)

                delta_w = randint(-w * 0.2, w * 0.2 + 1)
                delta_h = randint(-h * 0.2, h * 0.2 + 1)
                # random face box
                nx1 = int(max(x1 + w / 2 + delta_w - size / 2, 0))
                ny1 = int(max(y1 + h / 2 + delta_h - size / 2, 0))
                nx2 = nx1 + size
                ny2 = ny1 + size 

                if( nx2 > weight | ny2 > height):
                    continue               

                _box=[x1,y1,w,h]
                _box=np.array(_box).reshape(1,-1)     
                if(IoU(np.array([nx1,ny1,nx2,ny2]),_box)>0.65): 
                    facemark=np.zeros((5,2))
                    for i in range(5):
                        mark=((landmark[i][0]-nx1)/size,(landmark[i][1]-ny1)/size)
                        facemark[i]=mark  
                    img_list.append(cv2.resize(img[ny1:ny2,nx1:nx2,:], (img_size, img_size)))  
                    mark_list.append(facemark.reshape(10))

                    #mirro
                    mirro_mark=facemark.copy()
                    if(random.choice([0,1])):
                        img1,mirro_mark=flip(img[ny1:ny2,nx1:nx2,:],mirro_mark)
                        img_list.append(cv2.resize(img1, (img_size, img_size)))  
                        mark_list.append(mirro_mark.reshape(10))  

                    num=num-1
            for i in range(len(img_list)):

                if np.sum(np.where(mark_list[i] <= 0, 1, 0)) > 0:
                    continue

                if np.sum(np.where(mark_list[i] >= 1, 1, 0)) > 0:
                    continue

                cv2.imwrite(os.path.join(landmark_dir,'land_%d.jpg'%(land_idx)),img_list[i])
                mark=[str(_)for _ in mark_list[i]]
                f4.write(os.path.join(landmark_dir,'land_%d.jpg'%(land_idx)) +' -2 '+' '.join(mark)+'\n')
                land_idx=land_idx+1

    f4.close()    

if __name__=="__main__":

    img_size=48
    
    #change img_size to P R O net
    begin=time.time()

    lfw_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\prepare_data"
    pic_spilt_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\prepare_data\\trainImageList.txt"
    landmark_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\landmark"%(img_size)
    save_dir="E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d"%(img_size)

    if not os.path.exists(landmark_dir):
        os.makedirs(landmark_dir)    

    main()
    
    print(time.time()-begin)
    
#R_net 2320.5425374507904
#O_net 3131.904032945633

# gen_O48_TFRecord

In [ ]:
import tensorflow as tf 
import cv2
import random
import time

def main():
    
    for index,term in enumerate(terms):
        num=0
        print("%s start"%(term))
        with tf.python_io.TFRecordWriter("E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\%s_train.tfrecords"%(img_size,term)) as writer:
            with open(r'E:\friedhelm\object\face_detection_MTCNN\DATA\%d\%s.txt'%(img_size,term)) as readlines:
                readlines=[line.strip().split(' ') for line in readlines]
                random.shuffle(readlines)
                for i,line in enumerate(readlines):
                    if(i%50000==0):
                        print(i)
                    img=cv2.imread(line[0].replace('/','\\'))
                    img_raw = img.tobytes()
                    label=int(line[1])
                    roi=[0.0]*4               
                    landmark=[0.0]*10
                    if(len(line)==6):    
                        roi=[float(_) for _ in line[2:6]]   
                    if(len(line)==12):
                        landmark=[float(_) for _ in line[2:12]]                  
                    example = tf.train.Example(features=tf.train.Features(feature={
                        'img': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
                        "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
                        "roi": tf.train.Feature(float_list=tf.train.FloatList(value=roi)),
                        "landmark": tf.train.Feature(float_list=tf.train.FloatList(value=landmark)),
                    }))
                    writer.write(example.SerializeToString())  #序列化为字符串  
                    num+=1
                    if(num==base*scale[index]):
                        print("%s finish"%(term))
                        break

if __name__=="__main__":
    
    img_size=48
    #change img_size to P R O net
    terms=['neg_%d'%(img_size),'pos_%d'%(img_size),'par_%d'%(img_size),'land_%d'%(img_size)]
    scale=[3,1,1,2]
    base=180000

    begin=time.time()

    main()
    
    print(time.time()-begin)

#Pnet train_data               
#neg 645017
#par 507206
#pos 285560 
#land 584332
    
#Rnet train_data neg_pics 758503 in total,pos_pics 285017 in total,par_pics 572771 in total land 

#29951.208035707474

# train_O48model

In [ ]:
import tensorflow as tf
import time
from core.tool import read_multi_tfrecords,image_color_distort
from core.MTCNN_model import Pnet_model,Rnet_model,Onet_model
from train.train_tool import label_los,roi_los,landmark_los,cal_accuracy
import os



def train(image,label,roi,landmark,model,model_name):
    
    _label, _roi ,_landmark=model(image,batch)
    
    with tf.name_scope('output'):
        _label=tf.squeeze(_label,name='label')
        _roi=tf.squeeze(_roi,name='roi')
        _landmark=tf.squeeze(_landmark,name='landmark')
        
    _label_los=label_los(_label,label)
    _box_los=roi_los(label,_roi,roi)    
    _landmark_los=landmark_los(label,_landmark,landmark)
    
    function_loss=_label_los*ratio[0]+_box_los*ratio[1]+_landmark_los*ratio[2]

    tf.add_to_collection("loss", function_loss)
    loss_all=tf.get_collection('loss')
    
    with tf.name_scope('loss'):
        loss=tf.reduce_sum(loss_all)
        tf.summary.scalar('loss',loss) 
        
    opt=tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
    with tf.name_scope('accuracy'):
        train_accuracy=cal_accuracy(_label,label)
        tf.summary.scalar('accuracy',train_accuracy) 

    saver=tf.train.Saver()
    merged=tf.summary.merge_all() 
    
    images,labels,rois,landmarks=read_multi_tfrecords(addr,batch_size,img_size)   
    images=image_color_distort(images)
    
    with tf.Session() as sess:
        sess.run((tf.global_variables_initializer(),
                  tf.local_variables_initializer()))
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess,coord=coord)
        image_batch,label_batch,roi_batch,landmark_batch=sess.run([images,labels,rois,landmarks])
        
        writer_train=tf.summary.FileWriter('C:\\Users\\312\\Desktop\\',sess.graph)
        try:
            
            for i in range(1,train_step):
                
                image_batch,label_batch,roi_batch,landmark_batch=sess.run([images,labels,rois,landmarks])
                
                sess.run(opt,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch})
                if(i%100==0):
                    summary=sess.run(merged,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch})
                    writer_train.add_summary(summary,i) 
                if(i%1000==0):
                    print('次数',i)    
                    print('train_accuracy',sess.run(train_accuracy,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))
                    print('loss',sess.run(loss,{image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))               
                    print('time',time.time()-begin)
                    if(i%10000==0):
                        saver.save(sess,"E:\\friedhelm\\object\\face_detection_MTCNN\\%s\\%s_%d.ckpt"%(model_name,model_name,i))
        except  tf.errors.OutOfRangeError:
            print("finished")
        finally:
            coord.request_stop()
            writer_train.close()
        coord.join(threads)
    
def main(model):
    
    with tf.name_scope('input'):
        image=tf.placeholder(tf.float32,name='image')
        label=tf.placeholder(tf.int32,name='label')
        roi=tf.placeholder(tf.float32,name='roi')
        landmark = tf.placeholder(tf.float32,name='landmark')  

    train(image,label,roi,landmark,model,model_name)

if __name__=='__main__':
    
    img_size=48
    batch=448
    batch_size=[192,64,64,128]
    addr=["E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\neg_%d_train.tfrecords"%(img_size,img_size),
          "E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\pos_%d_train.tfrecords"%(img_size,img_size),
          "E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\par_%d_train.tfrecords"%(img_size,img_size),
          "E:\\friedhelm\\object\\face_detection_MTCNN\\DATA\\%d\\land_%d_train.tfrecords"%(img_size,img_size)]  

    model=Onet_model
    model_name="Onet_model"    
    train_step=100001
    learning_rate=0.001
    
    save_model_path="E:\\friedhelm\\object\\face_detection_MTCNN\\%s"%(model_name)
    if not os.path.exists(save_model_path):
        os.makedirs(save_model_path) 
        
    if(model_name=="Onet_model"):
        ratio=[1,0.5,1]
    else:
        ratio=[1,0.5,0.5]
    

    begin=time.time()        
    main(model)
# tensorboard --logdir=C:\\Users\\312\\Desktop\\

In [3]:
import numpy as np

a=[[1,2,3,4,5,6],[7,8,9,7,8,9]]
# np.vstack(a)
np.concatenate(a,axis=0)

array([1, 2, 3, 4, 5, 6, 7, 8, 9, 7, 8, 9])

In [5]:
a=np.array([1])
b=np.array([2])
c=[]
c.append(a)
c.append(b)

In [43]:
b=np.vstack([[1,2,3,4],[2,4,4,5]])

In [8]:
np.zeros((1,16))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [20]:
b

array([[1, 2, 3, 4],
       [2, 3, 4, 5]])

In [17]:
w=b[:,2]-b[:,0]

In [18]:
b[:,1]*w

array([4, 6])

In [37]:
np.where(b[:,1]-b[:,0]>1)[0]

array([1, 2], dtype=int64)

In [44]:
np.delete(b,np.where(b[:,1]-b[:,0]>1)[0],0)

array([[1, 2, 3, 4]])

In [48]:
b.shape

(2, 4)

In [50]:
c=np.zeros((5,2))

In [51]:
c[0]=(1,2)

In [53]:
c[[]]=(2,3)

In [54]:
c

array([[1., 2.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [ ]:
def pad(t_box,w,h):
    """
    used for pad t_box that out of range

    input : t_box; wide,high of img
    output: boundingbox after pad

    format of input  : 
        t_box : [x1,y1,x2,y2] 
        w       : int
        h       : int

    format of output : 
        t_box    : [x1,y1,x2,y2]
    """  
    xl_idx=np.where((t_box[:,0]<0))[0]  
    t_box[xl_idx]=0

    yl_idx=np.where((t_box[:,1]<0))[0]  
    t_box[yl_idx]=0  

    xr_idx=np.where((t_box[:,2]>w))[0]  
    t_box[xr_idx]=w-1

    yr_idx=np.where((t_box[:,3]>h))[0]  
    t_box[yr_idx]=h-1  

    return t_box



def calibrate_box(img,NMS_box,model_name="default"):
    """
    used for calibrating NMS_box

    input : boundingbox after nms, img
    output: score , boundingbox after calibrate , landmark_box

    format of input  : 
        NMS_box : -1*[x1,y1,x2,y2,score,offset_x1,offset_y1,offset_x2,offset_y2,5*(landmark_x,landmark_y)] 
        img          : np.array()

    format of output : 
        score_box    : list of score -1*[score]
        net_box      : list of box   -1*[face_x1,face_x2,face_y1,face_y2]
        landmark_box : list of box   -1*[5*(true_landmark_x,true_landmark_y)]
    """
    h,w,c=img.shape

    t_box=np.zeros((NMS_box.shape[0],4))
    boxes=np.vstack(NMS_box)

    bounding_box=boxes[:,0:4]
    score_box=boxes[:,4]
    delta_box=boxes[:,5:9]

    t_w=bounding_box[:,2]-bounding_box[:,0]+1
    t_h=bounding_box[:,3]-bounding_box[:,1]+1

    w_h=np.vstack([t_w,t_h,t_w,t_h])
    t_box=bounding_box+delta_box*w_h

    delete_idx=np.where((t_box[:,2]-t_box[:,0]<self.min_face_size)|(t_box[:,3]-t_box[:,1]<self.min_face_size))[0]
    t_box=np.delete(t_box,delete_idx,0)


    net_box=pad(t_box,w,h)

    if(model_name=="Onet"):
        boxes=np.delete(boxes,delete_idx,0)
        score_box=boxes[:,4]
        landmark_box=np.zeros((len(boxes),5,2))
        for i in range(5):
            landmark_box[:,i,:]=(boxes[:,9+i*2]*(t_box[:,2]-t_box[:,0])+t_box[:,0],boxes[:,9+i*2+1]*(t_box[:,3]-t_box[:,1])+t_box[:,1])   
    else: 
        score_box=np.delete(score_box,delete_idx,0)

    return score_box,net_box,landmark_box